# Index Rebalance Strategy

1. Trade stocks that are added / removed / up weights / down weights from the index

2. Names that are added / up weights to index => LONG

3. Names that are removed / down weights from index => SHORT

Intuition behind is large index fund or mutual fund or ETF usually tracks SP500, when names are added on SP500, they need to buy in their portfolios, </br>
whereas for those removed from SP500, they need to sell it. The momentum will drive the short term price movement post rebalancing


<b>TDDO List</b>

1. Backtest the strategy, find historical SP500 constituent data, we need to know each rebalance, what are the stocks removed or added

2. Based on backtest, find optimal holding periods

3. Optionally, how do we allcoate the capital among all stocks to achieve best growth

4. Need to find a real-time data source for upcoming rebalance (Dec), ideally we need to data before market open

<b>Sources</b>

1. Free: https://github.com/fja05680/sp500

2. Cheap: https://site.financialmodelingprep.com/developer/docs/historical-sp-500-companies-api/

<b>My Notes</b>

- SP500 can add and remove the constituent anytime, usually the news of adding / removing drive the momentum, not the day when the stocks are added

- SP500 rebalancing and add/remove can be at different dates

- <b>I'm only able to find the index addded / removed history, below is just backtesting add / removal only</b>

In [ ]:
%load_ext autoreload
%autoreload 2

from account.Futu import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
import warnings
from Utils import *
from utils.logging import get_logger
import yfinance as yf
from tqdm import tqdm

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200

logger = get_logger('Index Rebalance Drift')

# Backtest

Backtest the strategy by

- Take all historical constituent changes in SP500

- Define a holding periods

- Long when names are added whereas short when names are removed

- Assumptions:

    1. Assume the add/remove happens at start of trading

    2. we enter the position at open price whereas exit the position at close price after holding periods

In [ ]:
start_date = datetime(2018,1,1)
end_date = get_today()

In [ ]:
sp500_hist = get_sp500_constituent_hist()
sp500_hist = sp500_hist[sp500_hist['date'] >= start_date]
sp500_hist = sp500_hist[sp500_hist['date'] <= end_date]

logger.info('Date Range: {} - {}'.format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
logger.info('Number of SP500 constituent changes : {}'.format(len(sp500_hist)))

In [ ]:
symbols = list(sp500_hist['symbol'].unique())
raw = yf.download(symbols, interval="1d",auto_adjust=True, start=start_date - BDay(60), end=end_date + BDay(60))

In [86]:
entry_offset = -1
exit_offset = 1

In [91]:
def generate_position(entry_offset, exit_offset):

    position = pd.DataFrame(index=raw.index, columns=raw['Close'].columns).fillna(0)
    df_stats = []

    close = raw['Close']
    open = raw['Open']
    
    for _, row in sp500_hist.iterrows():        
        symbol = row['symbol']
        type = row['type']
        date = row['date']
        reason = row['reason']        
        
        entry_date = add_bday(date, entry_offset) 
        exit_date = add_bday(date, exit_offset)

        entry_px = open.loc[entry_date, symbol]        
        exit_px = close.loc[exit_date, symbol]

        pos = 1 if type == 'add' else -1
        return_ = pos * (exit_px - entry_px) / entry_px        
        position.loc[entry_date:exit_date, symbol] = pos

        row = {
            'date': date,
            'symbol': symbol,
            'type': type,
            'position': pos,
            'side': 'Long' if pos > 0 else 'Short',
            'entry_date': entry_date,
            'exit_date': exit_date,
            'entry_px': entry_px,
            'exit_px': exit_px,        
            'return': return_,
            'reason': reason,
            'entry_offset': entry_offset,
            'exit_offset': exit_offset,
        }
        df_stats.append(row)
        
    df_stats = pd.DataFrame(df_stats)
    return position, df_stats

position, df_stats = generate_position(entry_offset, exit_offset)

In [92]:
position_entry = position[position.abs().diff() == 1].fillna(0)
position_hold = position[position.abs().diff() == 0].fillna(0)

c2c_ret = raw['Close'] / raw['Close'].shift(1) - 1
o2c_ret = raw['Close'] / raw['Open'] - 1

ret = position_entry * o2c_ret + position_hold * c2c_ret
port_ret = ret.sum(axis=1)

performance_summary_plot(port_ret, strategy='Index Rebalance', benchmark=['^SPX'])

,Index Rebalance,^SPX
Measure,,
Cumulative Return,1.293600,1.772396
Annualized Return,0.113113,0.114909
Annualized Volatility,0.372708,0.204666
Annualized Sharpe Ratio,0.208645,0.388727
Maximum Drawdown,-0.472398,-0.339250


In [94]:
fig = make_subplots(1,2)
fig.update_layout(width=1000, height=500, title='Strategy Return vs Entry/Exit Day Offset on Rebal Day')

entry_offset_range = np.arange(-10,1)
exit_offset_range= np.arange(0,20,1)

entry_rets = []
exit_rets = []

c2c_ret = raw['Close'] / raw['Close'].shift(1) - 1
o2c_ret = raw['Close'] / raw['Open'] - 1

for o in entry_offset_range:
    position, temp = generate_position(o, exit_offset)

    position_entry = position[position.abs().diff() == 1].fillna(0)
    position_hold = position[position.abs().diff() == 0].fillna(0)
    ret = position_entry * o2c_ret + position_hold * c2c_ret
    port_ret = ret.sum(axis=1)
    entry_rets.append(cumulative_return(port_ret)[-1])

for o in exit_offset_range:
    position, temp = generate_position(entry_offset, o)

    position_entry = position[position.abs().diff() == 1].fillna(0)
    position_hold = position[position.abs().diff() == 0].fillna(0)
    ret = position_entry * o2c_ret + position_hold * c2c_ret
    port_ret = ret.sum(axis=1)
    exit_rets.append(cumulative_return(port_ret)[-1])        
    
fig.add_trace(go.Scatter(x=entry_offset_range, y=entry_rets, showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=exit_offset_range, y=exit_rets, showlegend=False), row=1, col=2)

fig['layout']['yaxis']['title'] = 'Cumulative Return (%)'
fig['layout']['xaxis']['title'] = f'Entry Days Before Rebal Days <br><sup>Assume Exit Days Offset is {exit_offset} on Rebal Days</sup>'

fig['layout']['yaxis2']['title'] = 'Cumulative Return (%)'
fig['layout']['xaxis2']['title'] = f'Exit Days Offset on Rebal Days <br><sup>Assume Entry Days Offset is {entry_offset} on Rebal Days</sup>'


fig.show()